# Experiment D

Ideas:

1) Implementar na classe do modelo o mapeamento entre empresas e `uid`.

2) Implementar como entrada uma empresa nova, conter mapeamento de valores.


- Author: Israel Oliveira [\[e-mail\]](mailto:'Israel%20Oliveira%20'<prof.israel@gmail.com>)

In [ ]:
%load_ext watermark

In [ ]:
import pandas as pd

In [ ]:
# Run this cell before close.
%watermark
%watermark --iversion
%watermark -b -r -g